Question 1

In [3]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models
client = QdrantClient("http://127.0.0.1:6333")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
query_to_embed = 'I just discovered the course. Can I join now?'
model_handle = 'jinaai/jina-embeddings-v2-small-en'
embedding_model = TextEmbedding(model_name=model_handle)
embeddings_list = list(embedding_model.embed(query_to_embed))
vector_q = embeddings_list[0]
print(len(vector_q))

512


In [5]:
min(vector_q)

np.float64(-0.11726373885183883)

Question 2

In [6]:
import numpy as np
np.linalg.norm(vector_q)

np.float64(1.0)

In [7]:
#Cosine similarity
doc = 'Can I still join the course after the start date?'
embeddings_list = list(embedding_model.embed(doc))
vector_doc = embeddings_list[0]
print(len(vector_doc))
print('Cosine Similarity is', vector_doc.dot(vector_q))

512
Cosine Similarity is 0.9008528895674548


Question 3

In [9]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [10]:
#Our query vector is query_to_embed = 'I just discovered the course. Can I join now?'
#We have to embed the text field
texts = [doc['text'] for doc in documents] #Extract text field
embeddings_doc_text = list(embedding_model.embed(texts))
similarity_score = []
for i in range(len(embeddings_doc_text)):
    similarity_score.append(embeddings_doc_text[i].dot(vector_q))
print('Index with highest cosine similarity score ', np.argmax(similarity_score))

Index with highest cosine similarity score  1


Question 4

In [11]:
full_text = [doc['question'] + ' ' + doc['text'] for doc in documents]
embeddings_doc_text = list(embedding_model.embed(full_text))
similarity_score = []
for i in range(len(embeddings_doc_text)):
    similarity_score.append(embeddings_doc_text[i].dot(vector_q))
print('Index with highest cosine similarity score ', np.argmax(similarity_score))

Index with highest cosine similarity score  0


Yes, Q3 and Q4 are different because in Q3 we only include the text/answer but in Q4 we have both answer/text and questions fields which provides a lot more information than embedding text alone.

Question 5

In [12]:
from fastembed import TextEmbedding
models_fastembed = TextEmbedding.list_supported_models()
smallest = min(models_fastembed, key=lambda x: x['dim'])
print(smallest)
smallest_model = smallest['model']
print(smallest_model)

{'model': 'BAAI/bge-small-en', 'sources': {'hf': 'Qdrant/bge-small-en', 'url': 'https://storage.googleapis.com/qdrant-fastembed/BAAI-bge-small-en.tar.gz', '_deprecated_tar_struct': True}, 'model_file': 'model_optimized.onnx', 'description': 'Text embeddings, Unimodal (text), English, 512 input tokens truncation, Prefixes for queries/documents: necessary, 2023 year.', 'license': 'mit', 'size_in_GB': 0.13, 'additional_files': [], 'dim': 384, 'tasks': {}}
BAAI/bge-small-en


Smallest dimensionality model is 'model': 'BAAI/bge-small-en' with 384 dimensions

Question 6

In [13]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [17]:
# Define the collection name
collection_name = "homework-rag"
client.delete_collection(collection_name=collection_name)
# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=384,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)

True

In [18]:
points = []

for i, doc in enumerate(documents):
    text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=text, model='BAAI/bge-small-en')
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)

In [19]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [20]:
from openai import OpenAI
import os
openai_client = OpenAI(api_key = os.getenv("OPENAI_API_KEY"))
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt
def llm(prompt):
    response = openai_client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [40]:
def vector_search(question):
    query_points = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=question,
            model='BAAI/bge-small-en' 
        ),
        limit=5,
        with_payload=True
    )
    results = []
    
    for point in query_points.points:
        results.append(point.payload)
    
    return results, query_points

In [41]:
def rag(query):
    search_results, points = vector_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer, points

In [42]:
answer, points = rag('I just discovered the course. Can I join now?')

In [43]:
print(answer)

Yes, you can join the course now. However, keep in mind that you won’t be able to submit some of the homeworks. You can still participate in the course, and if you complete 2 out of 3 course projects and review 3 peers’ projects by the deadline, you will be eligible for a certificate.


In [46]:
print(points)

points=[ScoredPoint(id=14, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'question': 'The course has already started. Can I still join it?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=6, version=0, score=0.86918855, payload={'text': 'Approximately 4 months, but may take more if you want to do some extra activities (an extra project, an article, etc)', 'section': 'General course-related questions', 'question': 'How long is the course?', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_va

The first returned highest score is 0.87